In [ ]:
# Code snippet for rendering animations in the docs
from IPython.display import HTML
import warnings
import matplotlib
matplotlib.rcParams['animation.embed_limit'] = 2**128
import matplotlib.pyplot as plt

def render_ani_func(f):
    om.utils.disable_interactive_backend_switching()
    plt.switch_backend('Agg')
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        ani = f()
    %matplotlib inline
    om.utils.enable_interactive_backend_switching()

    vid = HTML(ani.to_html5_video())
    plt.close('all')
    return vid

```{tip}
Download this tutorial as a {download}`Jupyter notebook <converted/motion_compensation.ipynb>`, or a {download}`python script <converted/motion_compensation.py>` with code cells.
```

# Tutorial 3: Motion Compensation and Ratiometry with Di-4-ANEPPS

In this tutorial, we will discuss the analysis of ratiometric optical mapping data showing the ventricular surface of a contracting heart. The heart was stained with Di-4-ANEPPS and imaged using an alternating green and blue illumination at 500fps as described in Kappadan et al. 2020. The tissue was illuminated in every odd frame with green light and in every even frame with blue light, respectively. This approach i sometimess referred to as 'excitation ratiometry'. After numerical motion tracking and stabilization, the green and blue videos are divided by each other to obtain a motion-stabilized ratiometric video. Using this method, both motion artifacts as well as illumination artifacts (which also lead to motion artifacts) can be significantly reduced. 

First, we load the video data, which is stored as a single file (`Example_05_Ratiometry.npy`) containing the two green and blue videos in an interleaved fashion (frame 1 = green, frame 2 = blue, frame 3 = green, etc.). Using optimap's `load_video()` function, we can specify to load only every 2nd frame and to start reading from the first or the second frame, respectively:

In [12]:
import optimap as om
# import monochrome as mc  # remove this if you don't have monochrome installed

#filename = om.utils.retrieve_example_data('Example_01_Sinus_Rabbit_Basler.npy')
filename = 'example_data/Example_05_Ratiometry.npy'
video_blue = om.load_video(filename, start_frame=0, step=2)
video_green = om.load_video(filename, start_frame=1, step=2)


video_green = om.video.rotate_left(video_green)
video_green = om.video.rotate_left(video_green)

video_blue = om.video.rotate_left(video_blue)
video_blue = om.video.rotate_left(video_blue)

Alternatively, one could load the video and manually split the video into the green and blue videos (`video_green = video[::2,:,:]` etc.).

When we play the two videos next to each other we notice that there is signal in only the green video but not in the blue video:

In [13]:
om.video.play2(video_green, video_blue, title1="green video with signal", title2="blue video without signal");

In [ ]:
render_ani_func(lambda: om.video.play2(video_green, 1, title="green video with signal", interval=20))

In [ ]:
warped = om.motion.motion_compensate(video_green, 5, ref_frame=40)
flows_nocontrast = om.motion.estimate_displacements(video, 40)
warped_nocontrast = om.motion.warp_video(video, flows_nocontrast)
om.video.playn([video, warped, warped_nocontrast],
               titles=["original video", "with contrast-enhancement", "w/o contrast-enhancement"], figsize=(8, 3.5));

In [ ]:
warped_ref0 = om.motion_compensate(video, contrast_kernel=5, ref_frame=0)
warped_ref40 = om.motion_compensate(video, contrast_kernel=5, ref_frame=40)
om.video.playn([video, warped_ref40, warped_ref0], titles=["original video", "compensated ref 40", "compensated ref 0"], figsize=(8, 3.5));

In [ ]:
warped_ref0 = om.motion_compensate(video, contrast_kernel=5, ref_frame=0)
warped_ref40 = om.motion_compensate(video, contrast_kernel=5, ref_frame=40)
render_ani_func(lambda: om.video.playn([video, warped_ref40, warped_ref0], titles=["original video", "compensated ref 40", "compensated ref 0"], interval=20, figsize=(8, 3.5)))

In [ ]:
contrast3 = om.motion.contrast_enhancement(video[:300], 3)
contrast5 = om.motion.contrast_enhancement(video[:300], 5)
contrast9 = om.motion.contrast_enhancement(video[:300], 9)
om.video.playn([contrast3, contrast5, contrast9],
               titles=["contrast kernel 3", "contrast kernel 5", "contrast kernel 9"],
               skip_frame=3,
               figsize=(8, 3.5));

In [ ]:
def f():
    contrast3 = om.motion.contrast_enhancement(video[:300], 3)
    contrast5 = om.motion.contrast_enhancement(video[:300], 5)
    contrast9 = om.motion.contrast_enhancement(video[:300], 9)
    return om.video.playn([contrast3, contrast5, contrast9], titles=["contrast kernel 3", "contrast kernel 5", "contrast kernel 9"], skip_frame=1, figsize=(8, 3.5))
render_ani_func(f)